In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, clear_output
from Simple_ESN import *

test_data = 0.5*(2*np.random.rand(6000)-1)
train_data = 0.5*(2*np.random.rand(6000)-1)

In [ ]:
################## input hebb multidim ##################
test_data = 0.5*(2*np.random.rand(6000,100,1)-1)
train_data = 0.5*(2*np.random.rand(6000,100,1)-1)
step= 1e-05
max_epochs= 600

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = ESN(Nu=100,r_density=0.1 , i_density=0.9)
  init_dim = compute_dim_state_space(esn,test_data)
  max_dim = init_dim ; max_ind=0

  for epoch in range(max_epochs):
    train_input(esn,train_data,step)
    act_dim = compute_dim_state_space(esn,test_data)
    if max_dim < act_dim : 
      max_dim = act_dim; max_ind=epoch
    if act_dim < max_dim:
      break

  
  dims[i] = [ init_dim, max_dim , epoch ]

dims = pd.DataFrame( np.array(dims).reshape(10,3) )
dims.columns=[' dim. iniziale ',' dim. massima ',' epoche ']
print("input hebb ")
print(dims.to_latex()) 
dims=dims.to_numpy()


print('media e varianza dimensionne iniziale:', np.mean(dims[:,0]))
print('media e varianza massimi dimensione: ',np.mean(dims[:,1]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(dims[:,0]-dims[:,1]), dims[:,0])*100 ),'%' )


9 [40.96728340158318, 49.13011423587118, 18]
input hebb 
\begin{tabular}{lrrr}
\toprule
{} &   dim. iniziale  &   dim. massima  &   epoche  \\
\midrule
0 &        40.242857 &       52.198596 &      27.0 \\
1 &        39.758373 &       51.517359 &      11.0 \\
2 &        36.752475 &       50.788130 &      17.0 \\
3 &        29.302211 &       49.931290 &       9.0 \\
4 &        44.080760 &       50.419935 &      20.0 \\
5 &        35.583606 &       50.767817 &      11.0 \\
6 &        39.653083 &       50.589876 &      18.0 \\
7 &        27.262286 &       47.743718 &      22.0 \\
8 &        40.967283 &       49.130114 &      18.0 \\
9 &        39.335025 &       50.823854 &      13.0 \\
\bottomrule
\end{tabular}

media e varianza dimensionne iniziale: 37.29379582473678
media e varianza massimi dimensione:  50.39106891353377
media e varianza incremento dimensione: 37.677004985562476 %


In [ ]:
################## input anti-hebb ##################
test_data = 0.5*(2*np.random.rand(6000,1)-1)
train_data = 0.5*(2*np.random.rand(6000,1)-1)
step= -1e-05
max_epochs= 2000

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = ESN(r_density=0.1 , i_density=0.9)
  init_dim = compute_dim_state_space(esn,test_data)
  max_dim = init_dim ; max_ind=0

  for epoch in range(max_epochs):
    train_input(esn,train_data,step)
    act_dim = compute_dim_state_space(esn,test_data)
    if max_dim < act_dim : 
      max_dim = act_dim; max_ind=epoch
    if act_dim < max_dim:
      break

  
  dims[i] = [ init_dim, max_dim , epoch ]

dims = pd.DataFrame( np.array(dims).reshape(10,3) )
dims.columns=[' dim. iniziale ',' dim. massima ',' epoche ']
print("input hebb ")
print(dims.to_latex()) 
dims=dims.to_numpy()


print('media e varianza dimensionne iniziale:', np.mean(dims[:,0]))
print('media e varianza massimi dimensione: ',np.mean(dims[:,1]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(dims[:,0]-dims[:,1]), dims[:,0])*100 ),'%' )


9 [4.82415272801682, 5.0901796981807905, 1422]
input hebb 
\begin{tabular}{lrrr}
\toprule
{} &   dim. iniziale  &   dim. massima  &   epoche  \\
\midrule
0 &         4.797185 &        5.432060 &     207.0 \\
1 &         5.817371 &        6.084805 &     129.0 \\
2 &         4.521884 &        4.864691 &      30.0 \\
3 &         5.577587 &        6.366486 &    1070.0 \\
4 &         5.512060 &        7.158419 &     128.0 \\
5 &         4.397951 &        4.888047 &     270.0 \\
6 &         5.958822 &        6.390488 &     627.0 \\
7 &         4.912145 &        5.250283 &     400.0 \\
8 &         4.824153 &        5.090180 &    1422.0 \\
9 &         6.939444 &        7.318436 &     197.0 \\
\bottomrule
\end{tabular}

media e varianza dimensionne iniziale: 5.325860283437229
media e varianza massimi dimensione:  5.884389525549277
media e varianza incremento dimensione: 10.56724353188871 %


In [ ]:
def train(self,train_x,train_y):
  c_state = self.compute_state
  s = np.array( list( map( c_state, train_x ) ) ) # shape(len(data),Nr,1)
  s = s.reshape( np.size(train_x), self.Nr ) # shape(len(data),Nr)
  d = train_y.T #shape(len(data),Ny)
  self.Wout = np.transpose( np.dot( np.linalg.pinv(s) , d ) )
  
def MC(esn,data):
  for d in data[:1000]:
    esn.compute_state(d) # washout

  c_out = esn.compute_output # funzione che calcola output
  m = np.array( list( map(c_out,data[1000:]) ) ).reshape(1000,esn.Ny) # matrice degli yk: uno per colonna

  v1 = np.var(data[1000-2*esn.Ny:])
  MC =(np.cov(m[:,0] , data[1000:])[0,0])**2 / (v1 * np.var(m[:,0])) + sum( (np.cov( m[:,k] , data[1000-k:-k])[0,0])**2 / ( v1 * np.var(m[:,k])) for k in range(1,esn.Ny) )
  if MC > 100 : MC=0
  return MC


In [ ]:
################## input anti-hebb ##################
train_data = 0.5*(2*np.random.rand(5000)-1)
test_data = 0.5*(2*np.random.rand(2000)-1)

train_x = train_data[1000:5000]
train_y = np.vstack( list( train_data[1000-k:5000-k] for k in range(200) ) )


step= -1e-05
max_epochs= 8000

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = ESN(r_density=0.1 , i_density=0.9,Ny=200)

  for d in train_data[:1000]:
    esn.compute_state(d) # washout
  train(esn,train_x,train_y) 
  
  init_dim = MC(esn,test_data)
  max_dim = init_dim ; max_ind=0

  for epoch in range(max_epochs):
    train_input(esn,train_data,step)

    for d in train_data[:1000]:
      esn.compute_state(d) # washout
    train(esn,train_x,train_y) 
    
    act_dim = MC(esn,test_data)
    if max_dim < act_dim : 
      max_dim = act_dim; max_ind=epoch
    if act_dim < max_dim:
      break

  
  dims[i] = [ init_dim, max_dim , epoch ]

dims = pd.DataFrame( np.array(dims).reshape(10,3) )
dims.columns=[' mem. iniziale ',' mem. massima ',' epoche ']
print("input hebb ")
print(dims.to_latex()) 
dims=dims.to_numpy()


print('media e varianza memoria iniziale:', np.mean(dims[:,0]))
print('media e varianza memoria finale: ',np.mean(dims[:,1]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(dims[:,0]-dims[:,1]), dims[:,0])*100 ),'%' )


9 [29.676959011149304, 75.85457149964965, 2951]
input hebb 
\begin{tabular}{lrrr}
\toprule
{} &   mem. iniziale  &   mem. massima  &   epoche  \\
\midrule
0 &        30.084557 &       75.892658 &    2998.0 \\
1 &        30.131871 &       73.450253 &    2862.0 \\
2 &        30.166266 &       71.664046 &    2806.0 \\
3 &        32.608000 &       80.853651 &    2976.0 \\
4 &        28.923962 &       70.264814 &    2870.0 \\
5 &        29.339632 &       71.260885 &    2848.0 \\
6 &        35.442230 &       81.000823 &    2951.0 \\
7 &        32.480934 &       76.081774 &    2855.0 \\
8 &        29.676959 &       75.854571 &    2951.0 \\
9 &        28.962725 &       70.051319 &    2850.0 \\
\bottomrule
\end{tabular}

media e varianza memoria iniziale: 30.781713405686865
media e varianza memoria finale:  74.63747935321086
media e varianza incremento dimensione: 142.7605773918104 %


In [ ]:
def ESP_Index(esn,data,P,T):
  esn.x = np.zeros( ( esn.Nr,1 ) ) # setto stato iniziale a 0
  # orbita x0. shape( len(data), (esn.N*esn.Nr), 1), ovvero un vettore di vettori colonna, ognuno dei quali rappresenta l'i-esimo stato (come concatenzione stati reservoir)
  s0 = np.array( list( esn.compute_state(d).reshape(-1,1) for d in data ) )  
  D = np.zeros( P ) # inizializzo vettore D
  for i in range( P ):
    esn.x = np.random.rand( esn.Nr, 1 ) # setto stato iniziale a vettore randomico
    si = np.array( list( esn.compute_state(d).reshape(-1,1) for d in data ) ) # orbita xi.
    d = np.zeros( np.size(data,axis=0) - T ) # per contenere distanze euclidee tra stati al t-esimo passo
    for t in range( T, np.size(data,axis=0) ):
      d[t-T] = np.linalg.norm( s0[t] - si[t] ) # d[t] contiene distanza tra stati al t-esimo passo, calcolati da stati iniziali diversi
    D[i] = np.mean( d ) # media distanze stati per due stati iniziali diversi
  return np.mean( D ) # a grandi linee media distanze stati per tanti stati iniziali diversi


In [ ]:
test_data = 0.5*(2*np.random.rand(2000)-1)
train_data = 0.5*(2*np.random.rand(5000)-1)
step= [1e-05,-1e-05]
max_epochs= 200

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = ESN(r_density=0.2 , i_density=0.8)
  init_dim = compute_dim_state_space(esn,test_data)
  max_dim = init_dim ; max_ind=0

  for epoch in range(max_epochs):
    train_both(esn,train_data,step)
  
  act_dim = compute_dim_state_space(esn,test_data)
  
  dims[i] = [ init_dim, act_dim , epoch, max(np.abs(np.linalg.eigvals(esn.W))), ESP_Index(esn,test_data,50,500) ]

dims = pd.DataFrame( np.array(dims).reshape(10,5) )
dims.columns=[' dim. iniziale ',' dim. massima ',' epoche ','raggio','esp']
print("input hebb ")
print(dims.to_latex()) 
dims=dims.to_numpy()


print('media e varianza dimensionne iniziale:', np.mean(dims[:,0]))
print('media e varianza massimi dimensione: ',np.mean(dims[:,1]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(dims[:,0]-dims[:,1]), dims[:,0])*100 ),'%' )
print('media e varianza raggio spttrale: ',np.mean(dims[:,3]))
print('media e varianza esp index: ',np.mean(dims[:,4]))


9 [4.152721017594193, 37.3473234151976, 199, 17.56710928909031, 10.434308924740735]
input hebb 
\begin{tabular}{lrrrrr}
\toprule
{} &   dim. iniziale  &   dim. massima  &   epoche  &     raggio &        esp \\
\midrule
0 &         7.367606 &       42.653293 &     199.0 &  23.906114 &  10.849835 \\
1 &         5.680220 &       38.185183 &     199.0 &  28.311116 &  10.549982 \\
2 &         5.973958 &       40.028667 &     199.0 &  32.496561 &  10.421124 \\
3 &         4.585408 &       37.497130 &     199.0 &  19.479300 &  10.213135 \\
4 &         5.430289 &       38.341956 &     199.0 &  46.308126 &  10.438342 \\
5 &         6.609010 &       38.603102 &     199.0 &  43.643482 &  10.803552 \\
6 &         5.342853 &       41.660723 &     199.0 &  16.335003 &  10.470115 \\
7 &         4.119084 &       42.185813 &     199.0 &  33.822030 &  11.173848 \\
8 &         4.152721 &       37.347323 &     199.0 &  17.567109 &  10.434309 \\
9 &         4.032015 &       42.191086 &     199.0 &  40.1487

In [ ]:
################## input anti-hebb ##################
train_data = 0.5*(2*np.random.rand(5000)-1)
test_data = 0.5*(2*np.random.rand(2000)-1)

train_x = train_data[1000:5000]
train_y = np.vstack( list( train_data[1000-k:5000-k] for k in range(200) ) )


step= [1e-05,-1e-05]
max_epochs= 400

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = ESN(r_density=0.8 , i_density=0.2,Ny=200)

  for d in train_data[:1000]:
    esn.compute_state(d) # washout
  train(esn,train_x,train_y) 
  
  init_dim = MC(esn,test_data)
  max_dim = init_dim ; max_ind=0

  for epoch in range(max_epochs):
    train_both(esn,train_data,step)

    for d in train_data[:1000]:
      esn.compute_state(d) # washout
    train(esn,train_x,train_y) 
    
    if epoch%10==0:
      act_dim = MC(esn,test_data)
      if max_dim < act_dim : 
        max_dim = act_dim; max_ind=epoch
      if act_dim < max_dim:
        break

  
  dims[i] = [ init_dim, max_dim , epoch, max(np.abs(np.linalg.eigvals(esn.W))), ESP_Index(esn,test_data,50,500) ]

dims = pd.DataFrame( np.array(dims).reshape(10,5) )
dims.columns=[' mem. iniziale ',' mem. massima ',' epoche ','r.s.','esp']
print("input hebb ")
print(dims.to_latex()) 
dims=dims.to_numpy()


print('media e varianza memoria iniziale:', np.mean(dims[:,0]))
print('media e varianza memoria finale: ',np.mean(dims[:,1]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(dims[:,0]-dims[:,1]), dims[:,0])*100 ),'%' )
print('media e varianza raggio spttrale: ',np.mean(dims[:,3]))
print('media e varianza esp index: ',np.mean(dims[:,4]))


9 [31.984969467476276, 34.97963297417404, 40, 1.0714457414174263, 1.0850499155492974]
input hebb 
\begin{tabular}{lrrrrr}
\toprule
{} &   mem. iniziale  &   mem. massima  &   epoche  &      r.s. &           esp \\
\midrule
0 &        29.429858 &       32.751712 &      40.0 &  1.145348 &  8.194531e-14 \\
1 &        32.256745 &       32.256745 &       0.0 &  0.900386 &  3.820480e-16 \\
2 &        31.022773 &       34.346050 &      50.0 &  1.173434 &  1.337198e-15 \\
3 &        31.903990 &       34.314800 &      40.0 &  1.106708 &  6.621290e-02 \\
4 &        34.269799 &       38.306894 &      50.0 &  1.200554 &  7.692917e-01 \\
5 &        32.097548 &       34.957454 &      40.0 &  1.079917 &  2.962616e-01 \\
6 &        33.504049 &       39.052478 &      50.0 &  1.219908 &  1.843160e+00 \\
7 &        30.326983 &       31.030424 &      30.0 &  0.970066 &  4.132980e-16 \\
8 &        31.984969 &       34.979633 &      40.0 &  1.071446 &  1.085050e+00 \\
9 &        32.208241 &       34.028171 

In [ ]:
################## input anti-hebb ##################
train_data = 0.5*(2*np.random.rand(5000)-1)
test_data = 0.5*(2*np.random.rand(2000)-1)

train_x = train_data[1000:5000]
train_y = np.vstack( list( train_data[1000-k:5000-k] for k in range(200) ) )


step= [1e-05,-1e-05]
max_epochs= 400

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = ESN(r_density=0.1 , i_density=0.5,Ny=200)
  
  init_dim = compute_dim_state_space(esn,test_data)
  max_dim = init_dim ; max_ind=0

  for epoch in range(max_epochs):
    train_both(esn,train_data,step)
    
    if epoch%1==0:
      act_dim = compute_dim_state_space(esn,test_data)
      if max_dim < act_dim : 
        max_dim = act_dim; max_ind=epoch
      if act_dim < max_dim:
        break

  
  dims[i] = [ init_dim, max_dim , epoch, max(np.abs(np.linalg.eigvals(esn.W))), ESP_Index(esn,test_data,50,500) ]

dims = pd.DataFrame( np.array(dims).reshape(10,5) )
dims.columns=[' mem. iniziale ',' mem. massima ',' epoche ','r.s.','esp']
print("input hebb ")
print(dims.to_latex()) 
dims=dims.to_numpy()


print('media e varianza memoria iniziale:', np.mean(dims[:,0]))
print('media e varianza memoria finale: ',np.mean(dims[:,1]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(dims[:,0]-dims[:,1]), dims[:,0])*100 ),'%' )
print('media e varianza raggio spttrale: ',np.mean(dims[:,3]))
print('media e varianza esp index: ',np.mean(dims[:,4]))


9 [4.392228296144336, 4.392228296144336, 0, 0.9011599220405523, 2.765112939864835e-16]
input hebb 
\begin{tabular}{lrrrrr}
\toprule
{} &   mem. iniziale  &   mem. massima  &   epoche  &      r.s. &           esp \\
\midrule
0 &         3.479049 &        3.479049 &       0.0 &  0.899690 &  2.710016e-16 \\
1 &         5.389898 &        5.389898 &       0.0 &  0.901261 &  5.237187e-16 \\
2 &         4.047387 &        4.047387 &       0.0 &  0.900469 &  3.069342e-16 \\
3 &         7.187966 &        7.187966 &       0.0 &  0.899982 &  4.529784e-16 \\
4 &         4.889157 &        4.889157 &       0.0 &  0.900337 &  3.960284e-16 \\
5 &         5.555873 &        5.555873 &       0.0 &  0.898236 &  4.415450e-16 \\
6 &         5.337587 &        5.337587 &       0.0 &  0.900398 &  3.392269e-16 \\
7 &         4.353841 &        4.353841 &       0.0 &  0.892448 &  2.392496e-16 \\
8 &         4.392228 &        4.392228 &       0.0 &  0.901160 &  2.765113e-16 \\
9 &         6.534318 &        6.534318

In [ ]:
################## input anti-hebb ##################
train_data = 0.5*(2*np.random.rand(5000)-1)
test_data = 0.5*(2*np.random.rand(2000)-1)

train_x = train_data[1000:5000]
train_y = np.vstack( list( train_data[1000-k:5000-k] for k in range(200) ) )


step= [1e-05,-1e-05]
max_epochs= 400

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = ESN(r_density=0.8 , i_density=0.2,Ny=200)
  
  init_dim = compute_dim_state_space(esn,test_data)
  max_dim = init_dim ; max_ind=0

  for epoch in range(max_epochs):
    train_both(esn,train_data,step)
    
    if epoch%1==0:
      act_dim = compute_dim_state_space(esn,test_data)
      if max_dim < act_dim : 
        max_dim = act_dim; max_ind=epoch
      if act_dim < max_dim:
        break

  
  dims[i] = [ init_dim, max_dim , epoch, max(np.abs(np.linalg.eigvals(esn.W))), ESP_Index(esn,test_data,50,500) ]

dims = pd.DataFrame( np.array(dims).reshape(10,5) )
dims.columns=[' mem. iniziale ',' mem. massima ',' epoche ','r.s.','esp']
print("input hebb ")
print(dims.to_latex()) 
dims=dims.to_numpy()


print('media e varianza memoria iniziale:', np.mean(dims[:,0]))
print('media e varianza memoria finale: ',np.mean(dims[:,1]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(dims[:,0]-dims[:,1]), dims[:,0])*100 ),'%' )
print('media e varianza raggio spttrale: ',np.mean(dims[:,3]))
print('media e varianza esp index: ',np.mean(dims[:,4]))


9 [5.505344921495875, 5.505344921495875, 0, 0.9007564922477845, 4.4132732117316707e-16]
input hebb 
\begin{tabular}{lrrrrr}
\toprule
{} &   mem. iniziale  &   mem. massima  &   epoche  &      r.s. &           esp \\
\midrule
0 &         5.114848 &        5.114848 &       0.0 &  0.900482 &  3.186523e-16 \\
1 &         5.641571 &        5.681501 &       4.0 &  0.903215 &  4.989416e-16 \\
2 &         6.096693 &        6.096693 &       0.0 &  0.899676 &  4.311517e-16 \\
3 &         5.629175 &        5.651293 &       1.0 &  0.901006 &  4.812980e-16 \\
4 &         5.982088 &        6.034070 &       3.0 &  0.893401 &  4.315408e-16 \\
5 &         5.365706 &        6.143020 &      36.0 &  1.075413 &  5.427388e-04 \\
6 &         4.522020 &        4.522020 &       0.0 &  0.899928 &  3.269526e-16 \\
7 &         4.835916 &        4.836771 &       1.0 &  0.898837 &  4.809438e-16 \\
8 &         5.505345 &        5.505345 &       0.0 &  0.900756 &  4.413273e-16 \\
9 &         5.844124 &        5.84412